In [ ]:
import openai
import os

from dotenv import find_dotenv, load_dotenv

from langsmith import traceable


_ = load_dotenv(find_dotenv())
openai.api_key = os.environ["OPENAI_API_KEY"]

MODEL_NAME = os.environ["MODEL_NAME"]

In [3]:
import uuid

thread_id = uuid.uuid4()

In [ ]:
import nest_asyncio

from openai import OpenAI
from typing import Dict, List
from utils import get_vector_db_retriever


openai_client = OpenAI()
nest_asyncio.apply()
retriever = get_vector_db_retriever()


@traceable(run_type="chain")
def retrieve_documents(question: str):
    return retriever.invoke(question)

@traceable(run_type="chain")
def generate_response(question: str, documents):
    formatted_docs = "\n\n".join(doc.page_content for doc in documents)
    rag_system_prompt = """
		You are an assistant for question-answering tasks. 
		Use the following pieces of retrieved context to answer the latest question in the conversation. 
		If you don't know the answer, just say that you don't know. 
		Use three sentences maximum and keep the answer concise.
    """
    messages = [
        {
            "role": "system",
            "content": rag_system_prompt
        },
        {
            "role": "user",
            "content": f"Context: {formatted_docs} \n\n Question: {question}"
        }
    ]
    return call_openai(messages)


@traceable(run_type="llm")
def call_openai(messages: List[Dict], model: str = MODEL_NAME, temperature: float = 0.0) -> str:
    return openai_client.chat.completions.create(
        model=model,
        messages=messages,
        temperature=temperature,
    )


@traceable(run_type="chain")
def langsmith_rag(question: str):
    documents = retrieve_documents(question)
    response = generate_response(question, documents)
    return response.choices[0].message.content


In [5]:
question = "How do I add metadata to a Trace?"
ai_answer = langsmith_rag(question, langsmith_extra={"metadata": {"thread_id": thread_id}})
ai_answer

'To add metadata to a trace in LangSmith, you can follow the instructions provided in the "Add metadata to your traces" section. This typically involves attaching key-value pairs to the trace during its creation or update process. For detailed steps, refer to the relevant documentation on the LangSmith website.'

In [6]:
question = "How can I add tags to a Trace?"
ai_answer = langsmith_rag(question, langsmith_extra={"metadata": {"thread_id": thread_id}})
ai_answer

'You can add tags to a trace in LangSmith by sending arbitrary metadata and tags along with the trace. Additionally, you can annotate a trace inline or send it to an annotation queue, where you can use feedback tags associated with your workspace. For more detailed instructions, refer to the Concepts page or the Querying Traces page.'